In [1]:
import requests
import json
from bs4 import BeautifulSoup

url = 'http://anom.archivesnationales.culture.gouv.fr/ir?ir=FRANOM_00019&q=&geogname=&date=&from=1608&to=1800&fbclid=IwAR2jhpSVAxCU4_Og2zKTawIqdYtOZSN1LzYrYi3ntYuwCPxVOZfzdST7q8Y&start=1&num=20000'
print("Loading...")
html = requests.get(url).content
tree = BeautifulSoup(html, 'html.parser')
print("Done")

links = tree.find_all("a", class_="res")[1:]
NAMES = [link.get_text() for link in links]
print(len(NAMES))

Loading...
Done
19251


In [4]:
import json
with open('officials_data.json', 'w') as f:
    json.dump(NAMES, f)

In [5]:
# Most common words

from collections import Counter

def most_common_words(sentences, n=500):
    words = [w for n in sentences for w in n.split()]
    counts = Counter(words)
    return counts.most_common(500)

most_common_words(NAMES)

[('de', 24075),
 ('à', 13458),
 ('la', 10191),
 ('au', 6867),
 ('du', 5401),
 ('des', 4778),
 ('régiment', 3938),
 ('Saint-Domingue', 3672),
 ('mort', 3539),
 ('et', 2723),
 ('en', 2709),
 ('de,', 2586),
 ("l'île", 2538),
 ('capitaine', 2492),
 ('soldat', 2216),
 ('Jean', 2046),
 ('natif', 1870),
 ('Martinique', 1853),
 ('aux', 1622),
 ('habitant', 1606),
 ('lieutenant', 1530),
 ('le', 1496),
 ('La', 1334),
 ('France', 1312),
 ('pour', 1288),
 ('troupes', 1251),
 ('Guadeloupe', 1200),
 ('dans', 1189),
 ('demande', 1173),
 ('François,', 1167),
 ('Saint-Domingue,', 1159),
 ('colonies', 1143),
 ('sa', 1097),
 ('Pierre,', 985),
 ('Roi', 985),
 ('Jean,', 930),
 ('les', 901),
 ('Guadeloupe,', 884),
 ('ancien', 852),
 ('Martinique,', 797),
 ('Louis', 739),
 ('Conseil', 728),
 ('milices', 710),
 ('Joseph,', 703),
 ('François', 691),
 ('1780', 688),
 ('supérieur', 685),
 ('1788', 685),
 ('Louis,', 683),
 ('Pierre', 648),
 ('commandant', 640),
 ('1787', 632),
 ('1784', 632),
 ('Le', 622),
 ('Cay

In [6]:
from collections import defaultdict

BOLD_START = '\033[1m'
BOLD_END = '\033[0m'
REGIONS = {
    'Caribbean': {
        'guadeloupe', 'martinique', 'saint-domingue', 
        'cap-français', 'port-au-prince', 'îles du vent',
        'la grenade', 'saint-louis'
    },
    'India': {'pondichéry', 'chandernagor', 'surate'},
    'Bourbon': {'bourbon', 'île de france'},
    'Canada': {'québec', 'canada', 'louisbourg', 'Montréal'},
    'Louisiana': {'louisiane', 'nouvelle-orléans'},
    'Guyana': {'cayenne'},
    'Senegal': {'sénégal', 'gorée'},
}

BLACKLIST = [
    # because Saint-Louis can be also Saint-Louis du Senegal
    {'saint-louis', 'sénégal', 'goreé'}
]
    

def find_index(needle, haystack):
    try:
        return haystack.lower().index(needle.lower())
    except ValueError:
        return -1

# TODO
# * remove boldface logic, return ordered keywords instead
# * set up graph, determine fields
# * export a JSON file that is graph-friendly
# * LAST STEP: delete by hand, 
#   storing in a BLACKLIST in case we need to regenerate the data

def get_regions(sentence):
    matching_regions = []
    matching_keywords = set()
    
    if "bourbonnais" in sentence.lower():
        # Ile Bourbon != Bourbonnais (village en France)
        return None, None
    
    for region, keywords in REGIONS.items():
        for kw in keywords:
            i_start = find_index(kw, sentence)
            if i_start > -1:
                matching_keywords.add(kw)
                matching_regions.append(region)
                break
    
    for _set in BLACKLIST:
        if matching_keywords.issubset(_set):
            return None, None

    if len(matching_regions) > 1:
        return matching_regions, matching_keywords
    return None, None



# Look into filtered data

In [16]:
CATEGORIES = {
    'military': ['régiment', 'capitaine', 'soldat', 'officier',
                'maréchal', 'brigadier', 'bataillon', 'infanterie',
                'troupes', 'caporal', 'déserteur', 'artillerie'],
    'officials': ['lieutenant', 'lieutenant', 'contrôleur', 'inspecteur',
                 'ordonnateur', 'sous-lieutenance', 'commissaire',
                 'gouverneur', 'écrivain', 'conseil', 'juge',
                 'garde-magasin'],
}
BY_PRECEDENCE = ['officials', 'military']

def get_category(name):
    for categ in BY_PRECEDENCE:
        for kw in CATEGORIES[categ]:
            if kw in name.lower():
                return categ
    return None


In [18]:
by_region = defaultdict(list)
FILTERED_NAMES = []

for name in NAMES:
    regions, keywords = get_regions(name)
#     if get_category(name) != 'officials':
#         continue
    if not regions:
        continue
    FILTERED_NAMES.append(name)
    by_region[' + '.join(regions)].append(name)
    
print("Total matches: {}\n".format(sum(len(v) for v in by_region.values())))
for reg, names in by_region.items():
    print("{} ({})".format(reg, len(names)))
#     print('================')
#     for n in names:
#         print(n, '\n')
    print()


# i_end = i_start + len(kw)
# sentence = (
#     sentence[:i_start] 
#     + BOLD_START
#     + sentence[i_start:i_end]
#     + BOLD_END
#     + sentence[i_end:]
# )

Total matches: 133

Caribbean + Canada (18)

Caribbean + Guyana (15)

Bourbon + Canada (5)

Caribbean + India + Bourbon (3)

Caribbean + Bourbon (21)

India + Bourbon (27)

Canada + Louisiana (4)

Caribbean + Louisiana (17)

Guyana + Senegal (2)

India + Canada (1)

India + Bourbon + Guyana (1)

Louisiana + Guyana (2)

Bourbon + Guyana (2)

Bourbon + Senegal (3)

Caribbean + Senegal (6)

India + Bourbon + Louisiana (1)

India + Senegal (2)

Caribbean + India (1)

Caribbean + Canada + Guyana (1)

Canada + Guyana (1)



## Add dates and export to CSV

In [14]:
import re

names_with_dates = []
for name in FILTERED_NAMES:
    match = re.search(r'(\d{4})(\/\d{4})?$', name)
    date = match.group(1) if match is not None else None
    names_with_dates.append([name, date])    

import csv
with open('officials_data_2.csv', 'w') as f:
    w = csv.writer(f)
    for entry in names_with_dates:
        w.writerow(entry)


In [17]:
from collections import Counter

not_categorized = []
by_category = Counter()

for name in FILTERED_NAMES:
    categorized = False
    by_category[get_category(name) or "other"] += 1
    if not categorized:
        not_categorized.append(name)

by_category            

Counter({'military': 116, 'officials': 133, 'other': 86})

In [10]:
import json
import io

def to_edges(bio, regions, keywords):
    edges = []
    for i in range(len(regions)-1):
        edges.append({
            'bio': bio,
            'from': regions[i],
            'to': regions[i+1],
            'category': get_category(bio) or 'other'
        })
    return edges

output = []
for name in FILTERED_NAMES:
    regions, keywords = get_regions(name)
    output += to_edges(name, regions, keywords)

with io.open('edges.json', 'w', encoding='utf8') as file_handle:
    json.dump(output, file_handle, ensure_ascii=False)